# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [106]:
# import libraries
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import pandas as pd
import numpy as np
import pickle
from sqlalchemy import create_engine
import pickle

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, accuracy_score

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Stew\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Stew\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Stew\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [3]:
# load data from database
engine = create_engine('sqlite:///../data/messagecategories.db')
df = pd.read_sql_table('messagecategories.db', engine)
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [4]:
df.related.value_counts()

1    19906
0     6122
2      188
Name: related, dtype: int64

In [96]:
X = df['message']
y = df.iloc[:,4:]

In [89]:
X.shape

(26216,)

In [90]:
y.shape

(26216, 36)

In [91]:
print(np.unique(y))

[0 1 2]


In [97]:
for col in y.columns:
    print(y[col].value_counts())

1    19906
0     6122
2      188
Name: related, dtype: int64
0    21742
1     4474
Name: request, dtype: int64
0    26098
1      118
Name: offer, dtype: int64
0    15356
1    10860
Name: aid_related, dtype: int64
0    24132
1     2084
Name: medical_help, dtype: int64
0    24903
1     1313
Name: medical_products, dtype: int64
0    25492
1      724
Name: search_and_rescue, dtype: int64
0    25745
1      471
Name: security, dtype: int64
0    25356
1      860
Name: military, dtype: int64
0    26216
Name: child_alone, dtype: int64
0    24544
1     1672
Name: water, dtype: int64
0    23293
1     2923
Name: food, dtype: int64
0    23902
1     2314
Name: shelter, dtype: int64
0    25811
1      405
Name: clothing, dtype: int64
0    25612
1      604
Name: money, dtype: int64
0    25918
1      298
Name: missing_people, dtype: int64
0    25341
1      875
Name: refugees, dtype: int64
0    25022
1     1194
Name: death, dtype: int64
0    22770
1     3446
Name: other_aid, dtype: int64
0    24511
1    

In [70]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [98]:
y.drop(columns='child_alone', inplace=True)

In [72]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,child_alone,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
y.dtypes

related                   int64
request                   int64
offer                     int64
aid_related               int64
medical_help              int64
medical_products          int64
search_and_rescue         int64
security                  int64
military                  int64
child_alone               int64
water                     int64
food                      int64
shelter                   int64
clothing                  int64
money                     int64
missing_people            int64
refugees                  int64
death                     int64
other_aid                 int64
infrastructure_related    int64
transport                 int64
buildings                 int64
electricity               int64
tools                     int64
hospitals                 int64
shops                     int64
aid_centers               int64
other_infrastructure      int64
weather_related           int64
floods                    int64
storm                     int64
fire    

In [8]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    '''
        PURPOSE: To clean raw text for analysis
        INPUT: text: a string value message
        OUTPUT: tokens: a tokenized, lemmatized array of words from text
        PROCESS:
            1. Replace any URLs in the messages with placeholder
            2. Remove punctuation and normalize case
            3. Tokenize
            4. Lemmatize words, removing stop words
    '''
    # Remove any urls
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
    # Remove punctuation and normalize case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Tokenize text
    tokens = word_tokenize(text)
    # Set up stop_words and lemmatizer elements
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    # Lemmatize and remove stop words
    clean_tokens = [lemmatizer.lemmatize(word).strip() for word in tokens if word not in stop_words]
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
# estimator
svc = SVC()

In [75]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([ 
            ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer())
        ]))
    ])),
#     ('clf', MultiOutputClassifier(estimator = AdaBoostClassifier))
    ('clf', MultiOutputClassifier(estimator = svc))
])

In [111]:
get_params(MultiOutputClassifier)

NameError: name 'get_params' is not defined

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [85]:
y_train.shape

(19662, 35)

In [100]:
for col in y_train.columns:
    print(y_train[col].value_counts())

1    14956
0     4565
2      141
Name: related, dtype: int64
0    16310
1     3352
Name: request, dtype: int64
0    19575
1       87
Name: offer, dtype: int64
0    11499
1     8163
Name: aid_related, dtype: int64
0    18072
1     1590
Name: medical_help, dtype: int64
0    18683
1      979
Name: medical_products, dtype: int64
0    19125
1      537
Name: search_and_rescue, dtype: int64
0    19306
1      356
Name: security, dtype: int64
0    19009
1      653
Name: military, dtype: int64
0    18407
1     1255
Name: water, dtype: int64
0    17472
1     2190
Name: food, dtype: int64
0    17918
1     1744
Name: shelter, dtype: int64
0    19355
1      307
Name: clothing, dtype: int64
0    19215
1      447
Name: money, dtype: int64
0    19443
1      219
Name: missing_people, dtype: int64
0    19005
1      657
Name: refugees, dtype: int64
0    18757
1      905
Name: death, dtype: int64
0    17093
1     2569
Name: other_aid, dtype: int64
0    18366
1     1296
Name: infrastructure_related, dtype: 

In [59]:
# y.unique()

In [52]:
df.iloc[:,1]

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
                               ...                        
26211    The training demonstrated how to enhance micro...
26212    A suitable candidate has been selected and OCH...
26213    Proshika, operating in Cox's Bazar municipalit...
26214    Some 2,000 women protesting against the conduc...
26215    A radical shift in thinking came about as a re...
Name: message, Length: 26216, dtype: object

In [53]:
df['message']

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
                               ...                        
26211    The training demonstrated how to enhance micro...
26212    A suitable candidate has been selected and OCH...
26213    Proshika, operating in Cox's Bazar municipalit...
26214    Some 2,000 women protesting against the conduc...
26215    A radical shift in thinking came about as a re...
Name: message, Length: 26216, dtype: object

In [63]:
# X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1],df.iloc[:,4:])

In [101]:
%%time

pipeline_fit = pipeline.fit(X_train, y_train)

Wall time: 33min 46s


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
y_pred = pipeline_fit.predict(X_test)

In [107]:
for i in range(len(y.columns)):
    print("Category: {}".format(y.columns.tolist()[i]))
    print(classification_report(y_test.iloc[:,i].values, y_pred[:,i]))
    print('Accuracy: {}'.format(accuracy_score(y_test.iloc[:,i].values, y_pred[:,i])))

Category: related
              precision    recall  f1-score   support

           0       0.77      0.39      0.51      1557
           1       0.83      0.96      0.89      4950
           2       0.00      0.00      0.00        47

    accuracy                           0.82      6554
   macro avg       0.53      0.45      0.47      6554
weighted avg       0.81      0.82      0.79      6554

Accuracy: 0.8191943851083308
Category: request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5432
           1       0.83      0.54      0.65      1122

    accuracy                           0.90      6554
   macro avg       0.87      0.76      0.80      6554
weighted avg       0.90      0.90      0.89      6554

Accuracy: 0.9025022886786696
Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

    accuracy      

c:\users\stew\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Category: buildings
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6244
           1       0.78      0.22      0.34       310

    accuracy                           0.96      6554
   macro avg       0.87      0.61      0.66      6554
weighted avg       0.95      0.96      0.95      6554

Accuracy: 0.9601769911504425
Category: electricity
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6437
           1       0.70      0.12      0.20       117

    accuracy                           0.98      6554
   macro avg       0.84      0.56      0.60      6554
weighted avg       0.98      0.98      0.98      6554

Accuracy: 0.9833689350015258
Category: tools
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6520
           1       0.00      0.00      0.00        34

    accuracy                           0.99      6554
   macro av

### 6. Improve your model
Use grid search to find better parameters. 

In [110]:
pipeline.get_params()

{'memory': None,
 'steps': [('features',
   FeatureUnion(transformer_list=[('text_pipeline',
                                   Pipeline(steps=[('vect',
                                                    CountVectorizer(tokenizer=<function tokenize at 0x0000029080312790>)),
                                                   ('tfidf',
                                                    TfidfTransformer())]))])),
  ('clf', MultiOutputClassifier(estimator=SVC()))],
 'verbose': False,
 'features': FeatureUnion(transformer_list=[('text_pipeline',
                                 Pipeline(steps=[('vect',
                                                  CountVectorizer(tokenizer=<function tokenize at 0x0000029080312790>)),
                                                 ('tfidf',
                                                  TfidfTransformer())]))]),
 'clf': MultiOutputClassifier(estimator=SVC()),
 'features__n_jobs': None,
 'features__transformer_list': [('text_pipeline',
   Pipeline(

In [112]:
parameters = {
    'clf__estimator__kernel': ['linear', 'rbf', 'poly'],
    'clf__estimator__degree': [1, 3, 6, 9]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [113]:
cv.fit(X_train, y_train)

KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.